In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

21/08/05 20:59:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/08/05 20:59:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/08/05 20:59:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/08/05 20:59:27 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
21/08/05 20:59:27 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
# Read in AP Data
df_ap = spark.read.csv('ap_data/18676_data_sample2.csv', header=True, inferSchema=True, sep=",", nullValue="")

In [3]:
from pyspark.sql.functions import isnan, when, count, col, trim
# Fix Column Name Formatting
df_ap = df_ap.toDF(*(c.replace('.', '') for c in df_ap.columns)) 
# Get null value count for each column
# df_ap.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ap.columns]).toPandas() 

In [4]:
import pyspark.sql.functions as F

# Drop Columns with all null values
def drop_null_columns(df):
    
    _df_length = df.count()
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v >= _df_length]
    df = df.drop(*to_drop)
    
    return df

df_ap = drop_null_columns(df_ap)

21/08/05 20:59:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
max_columns = [c for c in df_ap.columns if 'Max' in c]
average_columns = [c for c in df_ap.columns if 'Average' in c]
rate_average_columns = [c for c in df_ap.columns if 'Rate Average' in c]
std_columns = [c for c in df_ap.columns if 'Std Dev' in c]
total_columns = [c for c in df_ap.columns if 'Total' in c]
count_columns = [c for c in df_ap.columns if 'Count' in c]
count_average = [c for c in df_ap.columns if 'Count Average' in c]

In [6]:
columns_to_drop = total_columns + count_columns + rate_average_columns
[columns_to_drop.remove(c) for c in count_average]

df_ap = df_ap.drop(*columns_to_drop)

In [7]:
# Temporary Filtering for Outliers because of inaccurate data
bounds = {
    c: dict(
        zip(["q1", "q3"], df_ap.approxQuantile(c, [0.20, 0.80], 0))
    )
    for c in df_ap.columns[3:]
}

In [8]:
for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['lower'] = bounds[c]['q1'] - (iqr * 1.5)
    bounds[c]['upper'] = bounds[c]['q3'] + (iqr * 1.5)

In [9]:
df_ap = df_ap.select("*", *[F.when(F.col(c).between(bounds[c]['lower'], bounds[c]['upper']), 0).otherwise(1).alias(c+"_out") for c in df_ap.columns[3:]])

In [10]:
from functools import reduce
from operator import add

out_columns = [c for c in df_ap.columns if 'out' in c]
df_ap = df_ap.withColumn('sum', reduce(add, [col(x) for x in out_columns]))

In [11]:
df_ap = df_ap.drop(*out_columns)
df_ap_pd = df_ap.toPandas()
df_ap_pd.to_csv('18676_filtered_data_sample2.csv', index = False)

21/08/06 04:40:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 7207105 ms exceeds timeout 120000 ms
21/08/06 04:40:30 WARN SparkContext: Killing executors is not supported by current scheduler.
